# Colab config and dependencies

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Change the working directory
directory = '/content/drive/MyDrive/Tesi/lus-dl-framework'  # Replace this with the actual path of the directory
os.chdir(directory)

Mounted at /content/drive


In [ ]:
#!pip install --quiet cloud-tpu-client torch transformers datasets pytorch-lightning https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
!pip install --quiet torch transformers datasets pytorch-lightning torchmetrics pytorch-lightning[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Experiment

## TensorBoard

In [1]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

ModuleNotFoundError: No module named 'tensorboard'

## Script launch

In [4]:
!python main.py --num_workers 8\
                    --accumulate_grad_batches 4\
                    --max_epochs 2\
                    --accelerator "mps" \
                    --batch_size 90

Traceback (most recent call last):
  File "/Users/andry/Documents/GitHub/lus-dl-framework/main.py", line 6, in <module>
    import h5py
ModuleNotFoundError: No module named 'h5py'


In [ ]:
"""!python main_vit.py fit \
                    --trainer.callbacks+=EarlyStopping \
                    --trainer.callbacks.patience=3 \
                    --trainer.callbacks.monitor='validation_loss'\
                    --trainer.callbacks.mode='min' \
                    --trainer.callbacks+=LearningRateMonitor \
                    --trainer.callbacks.logging_interval=epoch \
                    --trainer.accelerator="gpu" \
                    --trainer.accumulate_grad_batches=4 \
                    --trainer.max_epochs=2 \
                    --trainer.strategy="ddp" \
                    --trainer.enable_progress_bar=True \
                    --trainer.precision=16
"""

'!python main_vit.py fit                     --trainer.callbacks+=EarlyStopping                     --trainer.callbacks.patience=3                     --trainer.callbacks.monitor=\'validation_loss\'                    --trainer.callbacks.mode=\'min\'                     --trainer.callbacks+=LearningRateMonitor                     --trainer.callbacks.logging_interval=epoch                     --trainer.accelerator="gpu"                     --trainer.accumulate_grad_batches=4                     --trainer.max_epochs=2                     --trainer.strategy="ddp"                     --trainer.enable_progress_bar=True                     --trainer.precision=16\n'